# test GLEE


In [1]:
import sys
sys.path.append('/ssd/xiaxinyuan/code/w61-grutopia/vlmaps/vlmaps/GLEE')
print(sys.path)

['', '/home/xiaxinyuan/.conda/envs/isaacsim/lib/python3.10/site-packages', '/ssd/xiaxinyuan/code/w61-grutopia/vlmaps/application_my/glee', '/home/xiaxinyuan/.conda/envs/isaacsim/lib/python310.zip', '/home/xiaxinyuan/.conda/envs/isaacsim/lib/python3.10', '/home/xiaxinyuan/.conda/envs/isaacsim/lib/python3.10/lib-dynload', '/home/xiaxinyuan/.local/lib/python3.10/site-packages', '__editable__.omni_isaac_lab-0.22.1.finder.__path_hook__', '__editable__.omni_isaac_lab_assets-0.1.3.finder.__path_hook__', '__editable__.omni_isaac_lab_tasks-0.10.0.finder.__path_hook__', '/ssd/xiaxinyuan/code/w61-grutopia', '/home/xiaxinyuan/.conda/envs/isaacsim/lib/python3.10/site-packages/setuptools/_vendor', '/ssd/xiaxinyuan/code/w61-grutopia/vlmaps/vlmaps/GLEE']


In [2]:
import cv2
import matplotlib.pyplot as plt
def display_image_in_notebook(image_path):
    # 读取图像
    image = cv2.imread(image_path)
    
    # 检查图像是否成功读取
    if image is None:
        print(f"Failed to load image from {image_path}")
        return
    
    # 将图像从 BGR 转换为 RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # 在 Jupyter Notebook 中显示图像
    plt.imshow(image_rgb)
    plt.axis('off')  # 关闭坐标轴
    plt.show()

In [3]:
from vlmaps.application_my.glee.glee_detector import initialize_glee, glee_segmentation, visualize_segmentation, visualize_detection
img_file = "logs/sample_episodes_safe/s8pcmisQ38h/id_37/rgb/pano_camera_0_image_step_269.png"
img = cv2.imread(img_file)
model = initialize_glee()
bbox_pred, mask_pred, classes, scores = glee_segmentation(img, model)

seg = visualize_segmentation(img, classes, mask_pred)
det = visualize_detection(img, classes, bbox_pred)

display_image_in_notebook(seg)
display_image_in_notebook(det)


/ssd/xiaxinyuan/code/w61-grutopia/vlmaps/vlmaps/GLEE/glee/models/pixel_decoder/maskdino_encoder.py:383: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled=False)
/ssd/xiaxinyuan/code/w61-grutopia/vlmaps/vlmaps/GLEE/glee/models/transformer_decoder/dino_decoder.py:249: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled=False)
/ssd/xiaxinyuan/code/w61-grutopia/vlmaps/application_my/glee/glee_detector.py:35: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will

FileNotFoundError: [Errno 2] No such file or directory: 'GLEE/weights/GLEE_SwinL_Scaleup10m.pth'

In [ ]:
# inference: output the feature of each pixel





In [ ]:
# index: query the feature of each pixel with the index
# refer to vlmaps.vlmaps.utils.clip_utils:
scores_mat = get_lseg_score(
        self.clip_model,
        [language_desc],
        self.grid_feat,
        self.clip_feat_dim,
        use_multiple_templates=True,
        add_other=True,
    ) 

def get_lseg_score(
    clip_model,
    landmarks: list,
    lseg_map: np.array,
    clip_feat_dim: int,
    use_multiple_templates: bool = False,
    avg_mode: int = 0,
    add_other=True,
):
    """
    Inputs:
        landmarks: a list of strings that describe the landmarks
        lseg_map: a numpy array with shape (h, w, clip_dim) or (N, clip_dim)
        avg_mode: this is for multiple template. 0 for averaging features, 1 for averaging scores
    Return:
        scores_list: (h, w, class_num) storing the score for each location for each class
    """
    if lseg_map is None:
        print("lseg_map is empty")
        return np.zeros((2,clip_feat_dim)) # (map_size.flatten(),clip_feat_dim)
    landmarks_other = landmarks
    if add_other and landmarks_other[-1] != "other":
        landmarks_other = landmarks + ["other"]

    if use_multiple_templates:
        mul_tmp = multiple_templates.copy()
        multi_temp_landmarks_other = [x.format(lm) for lm in landmarks_other for x in mul_tmp]
        text_feats = get_text_feats(multi_temp_landmarks_other, clip_model, clip_feat_dim)

        # average the features
        if avg_mode == 0:
            text_feats = text_feats.reshape((-1, len(mul_tmp), text_feats.shape[-1]))
            text_feats = np.mean(text_feats, axis=1)

        map_feats = lseg_map.reshape((-1, lseg_map.shape[-1]))
        map_feats = map_feats / np.linalg.norm(map_feats, axis=1, keepdims=True)
        scores_list = map_feats @ text_feats.T

        # average the features
        if avg_mode == 1:
            scores_list = scores_list.reshape((-1, len(landmarks_other), len(mul_tmp)))
            scores_list = np.mean(scores_list, axis=2)
    else:
        text_feats = get_text_feats(landmarks_other, clip_model, clip_feat_dim)

        map_feats = lseg_map.reshape((-1, lseg_map.shape[-1]))

        map_feats = map_feats / np.linalg.norm(map_feats, axis=1, keepdims=True)
        scores_list = map_feats @ text_feats.T            
        

    return scores_list


pix_feats, pix_mask = get_glee_feat(
    self.glee_model,
    rgb,
    labels,
    self.glee_transform,
    self.device,
    self.crop_size,
    self.base_size,
    self.norm_mean,
    self.norm_std,
    vis = True,
    save_path = self.segmentation_dir + f"/{step}.jpg"
)
# pix_feats: [1,feat,h,w]; pix_mask:[480,640](segmentation result)
